# Verbessertes Link-Replacement 

### Schritte 

1. JSON DATEI laden: 
    * datei lesen und Liste Verarbeiten
2. Überprüfung der Hauptlinks:
    * jeder Eintrag in der Liste wird überprüft 
    * wenn der link nicht funktioniert, wird ein Erstaz aus der Liste genommen, wenn dieser funktioniert als neuer Hauptlink eingetragen. 
    * Funktioniert kein alternativer Link, bleibt der Eintrag ohne Hauptlink, und das Problem wird geloggt.
3. Fallback-Logik:
    *  Der erste funktionierende Link aus der Liste alternatives ersetzt den Eintrag in main
    *  Funktioniert kein alternativer Link, bleibt der Eintrag ohne Hauptlink, und das Problem wird geloggt
4. JSON-Datei aktualisieren: 
    * Nach der Überprüfung wird die Datei mit den aktualisierten Links überschrieben.
5. Statistik und Logging:
    * Protokolliert die Anzahl der überprüften Links, ersetzten Hauptlinks und fehlerhaften Links.
    

In [ ]:
# Example JSON-Datei 
# additional infromaton can be added 
[
    {
        "main": "https://twitter.com/example/image1.jpg",
        "alternatives": [
            "https://backup.com/image1.jpg",
            "https://anotherbackup.com/image1.jpg"
        ]
    },
    {
        "main": "https://twitter.com/example/image2.jpg",
        "alternatives": [
            "https://backup.com/image2.jpg"
        ]
    }
]


### Regelmäßige Ausführung 
* using Cronejobs 

* Beispiel: 0 0 1 * * /usr/bin/python3 /path/to/script.py


### CODE 

In [8]:
import json
import requests
import logging
from datetime import datetime

In [9]:
# Logging konfigurieren
logging.basicConfig(filename="link_check.log", level=logging.INFO, format="%(asctime)s - %(message)s")

# Pfad zur JSON-Datei
JSON_FILE = "image_links.json"

In [10]:
def load_json(file_path):
    #Lädt die JSON-Datei
    with open(file_path, "r", encoding="utf-8") as f:
        return json.load(f)

In [11]:
def save_json(data, file_path):
    #Speichert die aktualisierte JSON-Datei
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)


In [12]:
def check_link(url):
    #Überprüft, ob ein Link erreichbar ist
    try:
        response = requests.head(url, timeout=5)
        return response.status_code == 200
    except requests.RequestException:
        return False


In [13]:
def process_links(data):
    #Überprüft und ersetzt ungültige Links
    replaced = 0
    broken = 0

    for entry in data:
        main_link = entry["main"]
        alternatives = entry.get("alternatives", [])

        if not check_link(main_link):
            logging.info(f"Ungültiger Hauptlink: {main_link}")
            found_replacement = False

            # Suche nach einer funktionierenden Alternative
            for alt in alternatives:
                if check_link(alt):
                    logging.info(f"Ersetze {main_link} durch Alternative {alt}")
                    entry["main"] = alt
                    replaced += 1
                    found_replacement = True
                    break
            
            if not found_replacement:
                logging.warning(f"Kein Ersatz gefunden für {main_link}")
                broken += 1

    return replaced, broken

In [ ]:
# alternative: for item in data:
        main_link = item['main']
        alternatives = item['alternatives']

        valid_alternatives = validate_links(alternatives)
        
        if valid_alternatives:
            updated_data.append({
                "main": main_link,
                "alternatives": valid_alternatives
            })
            replaced_count += 1
        else:
            print(f"Kein Ersatz gefunden für {main_link}")
            invalid_count += 1

    with open(file_path, 'w') as file:
        json.dump(updated_data, file, indent=4)

    print(f"Überprüfung abgeschlossen: {replaced_count} ersetzt, {invalid_count} ungültig.")


In [14]:
if __name__ == "__main__":
    # Schritt 1: JSON-Datei laden
    links_data = load_json(JSON_FILE)
    
    # Schritt 2: Links überprüfen und ersetzen
    replaced_count, broken_count = process_links(links_data)
    
    # Schritt 3: JSON-Datei aktualisieren
    save_json(links_data, JSON_FILE)
    
    # Schritt 4: Statistik ausgeben
    logging.info(f"Überprüfung abgeschlossen: {replaced_count} ersetzt, {broken_count} ungültig.")
    print(f"Überprüfung abgeschlossen: {replaced_count} ersetzt, {broken_count} ungültig.")

FileNotFoundError: [Errno 2] No such file or directory: 'update_automation/image_links.json'

### Was ist Cronjobs: Infos and Overview

1. Def: zeitgesteuerte Aufgaben, die auf Unix- oder Linux-Systemen mithilfe des Dienstprogramme cron ausgeführt werden. Ermöglicht Programme, Skripte oder Befehle automatisch zu vorgegebenen Zeiten oder n regelmäßigen Intervallen auszuführen.

2. Anwendungsbeispiele: 
    * Regelmäßige Sicherungen, Automatsiches Updates, Dateiüberwachung, Berichte erstellen, Skripte ausführen 



####  HOW TO 

1. Grundlegender Aufbau einer Crontab-Zeile
    + * * * * * /pfad/zum/befehl
    + Minute Stunde Tag Monat Wochentag / 0-59 0-23 1-31 1-12 0-7
    



In [ ]:
# täglich um Mitternacht 
# 0 0 * * * /usr/bin/python3 /home/user/script.py
# jede Stunde 
# 0 * * * * /usr/bin/python3 /home/user/script.py
# alle 10 minuten 
# */10 * * * * /usr/bin/python3 /home/user/script.py
# montags um 09:00 Uhr 
# 0 9 * * 1 /usr/bin/python3 /home/user/script.py


### Verwaltung 

* Cronejobs anzeigen = crontab -1 aktueller Benutzer 
* Cronjobs bebarbeiten = crontab -e Crontab-Datei bearbieten und neue Cronjobs hinzufügen 
* Cronejobs entfernen = einzelne einträge können durch köschen der Zeile in crontab -e entfernt werden 
* starten und überprüfen:
    * sudo service cron start
    * sudo service cron status
